In [ ]:
# hdfs : http://localhost:9870/explorer.html#/danny/data/data/flight-data/csv
# yarn :http://localhost:8188/cluster/apps
# Spark : http://localhost:4040/

In [1]:
from pyspark import SparkContext
sc =SparkContext()


22/09/12 21:59:34 WARN Utils: Your hostname, sunghwanui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.8 instead (on interface en0)
22/09/12 21:59:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/12 21:59:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/12 21:59:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [3]:
rdd = sc.textFile("hdfs://localhost:9000/danny/data/test.txt")

In [4]:
rdd.count()

595

In [5]:
from pyspark.sql import SparkSession, SQLContext

In [6]:
spark=SparkSession.builder.appName('company').getOrCreate()


In [7]:
spark

In [8]:
sqlContext = SQLContext(spark)

/Users/sunghwanki/.pyenv/versions/3.7.5/envs/python37/lib/python3.7/site-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [9]:
data = sqlContext.createDataFrame([(1.449015065E9, 1.449015065E9), (0.0,1.449015065E9),(1.449015065E9,0.0),(0.0,0.0)],
                                  ["modified", "created"])

In [10]:
data.show()

+-------------+-------------+
|     modified|      created|
+-------------+-------------+
|1.449015065E9|1.449015065E9|
|          0.0|1.449015065E9|
|1.449015065E9|          0.0|
|          0.0|          0.0|
+-------------+-------------+



In [11]:
data.printSchema()

root
 |-- modified: double (nullable = true)
 |-- created: double (nullable = true)



In [12]:
data.select(data.modified).show()

+-------------+
|     modified|
+-------------+
|1.449015065E9|
|          0.0|
|1.449015065E9|
|          0.0|
+-------------+



In [13]:
myRange = spark.range(100).toDF("number")
myRange.take(10)

[Row(number=0),
 Row(number=1),
 Row(number=2),
 Row(number=3),
 Row(number=4),
 Row(number=5),
 Row(number=6),
 Row(number=7),
 Row(number=8),
 Row(number=9)]

In [14]:
# myRange.show()
diviBy2 = myRange.where("number % 2 = 0")
diviBy2.show()

+------+
|number|
+------+
|     0|
|     2|
|     4|
|     6|
|     8|
|    10|
|    12|
|    14|
|    16|
|    18|
|    20|
|    22|
|    24|
|    26|
|    28|
|    30|
|    32|
|    34|
|    36|
|    38|
+------+
only showing top 20 rows



In [15]:
flightData2015 = spark.read.option("inferSchema", "true").option("header", "true").csv("hdfs://localhost:9000/danny/data/data/flight-data/csv/2015-summary.csv")

In [16]:
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [17]:
flightData2015.show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
+-----------------+-------------------+-----+
only showing top 3 rows



In [18]:
flightData2015.sort("count").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#46 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#46 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#97]
      +- FileScan csv [DEST_COUNTRY_NAME#44,ORIGIN_COUNTRY_NAME#45,count#46] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://localhost:9000/danny/data/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [35]:
flightData2015.sort("count").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Estonia|    1|
|              Kosovo|      United States|    1|
|              Zambia|      United States|    1|
|       United States|   Papua New Guinea|    1|
|               Malta|      United States|    1|
|       United States|          Gibraltar|    1|
|            Suriname|      United States|    1|
|       United States|            Croatia|    1|
|            Djibouti|      United States|    1|
|        Burkina Faso|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|             Cyprus|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
|              Cyprus|      United States|    1|
|       United States|          Lithuania|    1|
|       United States|           Bulgaria|    1|
|       United State

In [34]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [22]:
flightData2015.sort("count").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#46 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#46 ASC NULLS FIRST, 5), ENSURE_REQUIREMENTS, [id=#113]
      +- FileScan csv [DEST_COUNTRY_NAME#44,ORIGIN_COUNTRY_NAME#45,count#46] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://localhost:9000/danny/data/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [36]:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [37]:
sqlWay = spark.sql("""
select dest_country_name, count(1) from flight_data_2015 group by dest_country_name
""")

In [40]:
sqlWay.show()

+--------------------+--------+
|   dest_country_name|count(1)|
+--------------------+--------+
|             Moldova|       1|
|             Bolivia|       1|
|             Algeria|       1|
|Turks and Caicos ...|       1|
|            Pakistan|       1|
|    Marshall Islands|       1|
|            Suriname|       1|
|              Panama|       1|
|         New Zealand|       1|
|             Liberia|       1|
|             Ireland|       1|
|              Zambia|       1|
|            Malaysia|       1|
|               Japan|       1|
|    French Polynesia|       1|
|           Singapore|       1|
|             Denmark|       1|
|               Spain|       1|
|             Bermuda|       1|
|            Kiribati|       1|
+--------------------+--------+
only showing top 20 rows



In [41]:
sqlWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[dest_country_name#44], functions=[count(1)])
   +- Exchange hashpartitioning(dest_country_name#44, 5), ENSURE_REQUIREMENTS, [id=#306]
      +- HashAggregate(keys=[dest_country_name#44], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#44] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://localhost:9000/danny/data/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [42]:
dataFrameWay = flightData2015.groupBy("dest_country_name").count()
dataFrameWay.show()

+--------------------+-----+
|   dest_country_name|count|
+--------------------+-----+
|             Moldova|    1|
|             Bolivia|    1|
|             Algeria|    1|
|Turks and Caicos ...|    1|
|            Pakistan|    1|
|    Marshall Islands|    1|
|            Suriname|    1|
|              Panama|    1|
|         New Zealand|    1|
|             Liberia|    1|
|             Ireland|    1|
|              Zambia|    1|
|            Malaysia|    1|
|               Japan|    1|
|    French Polynesia|    1|
|           Singapore|    1|
|             Denmark|    1|
|               Spain|    1|
|             Bermuda|    1|
|            Kiribati|    1|
+--------------------+-----+
only showing top 20 rows



In [43]:
dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[dest_country_name#44], functions=[count(1)])
   +- Exchange hashpartitioning(dest_country_name#44, 5), ENSURE_REQUIREMENTS, [id=#364]
      +- HashAggregate(keys=[dest_country_name#44], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#44] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://localhost:9000/danny/data/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [65]:
from pyspark.sql.functions import max

flightData2015.select(max("count")).show()

+----------+
|max(count)|
+----------+
|    370002|
+----------+



In [66]:
maxSql = spark.sql("""
select dest_country_name, sum(count) as destination_total
from flight_data_2015
group by dest_country_name
order by sum(count) desc
limit 5
""")

In [67]:
maxSql.show()

+-----------------+-----------------+
|dest_country_name|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [69]:
from pyspark.sql.functions import desc
flightData2015.groupBy("dest_country_name").sum("count").withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total")).limit(5).show()

+-----------------+-----------------+
|dest_country_name|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [50]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]
columns = ["firstname","lastname","country","state"]
df = spark.createDataFrame(data = data, schema = columns)
df.show(truncate=False)

+---------+--------+-------+-----+
|firstname|lastname|country|state|
+---------+--------+-------+-----+
|James    |Smith   |USA    |CA   |
|Michael  |Rose    |USA    |NY   |
|Robert   |Williams|USA    |CA   |
|Maria    |Jones   |USA    |FL   |
+---------+--------+-------+-----+



In [51]:
df.show(truncate=True)

+---------+--------+-------+-----+
|firstname|lastname|country|state|
+---------+--------+-------+-----+
|    James|   Smith|    USA|   CA|
|  Michael|    Rose|    USA|   NY|
|   Robert|Williams|    USA|   CA|
|    Maria|   Jones|    USA|   FL|
+---------+--------+-------+-----+



In [52]:
df.select("firstname","lastname").show()
df.select(df.firstname,df.lastname).show()
df.select(df["firstname"],df["lastname"]).show()

+---------+--------+
|firstname|lastname|
+---------+--------+
|    James|   Smith|
|  Michael|    Rose|
|   Robert|Williams|
|    Maria|   Jones|
+---------+--------+

+---------+--------+
|firstname|lastname|
+---------+--------+
|    James|   Smith|
|  Michael|    Rose|
|   Robert|Williams|
|    Maria|   Jones|
+---------+--------+

+---------+--------+
|firstname|lastname|
+---------+--------+
|    James|   Smith|
|  Michael|    Rose|
|   Robert|Williams|
|    Maria|   Jones|
+---------+--------+



In [53]:
#By using col() function
from pyspark.sql.functions import col
df.select(col("firstname"),col("lastname")).show()

#Select columns by regular expression
df.select(df.colRegex("`^.*name*`")).show()

+---------+--------+
|firstname|lastname|
+---------+--------+
|    James|   Smith|
|  Michael|    Rose|
|   Robert|Williams|
|    Maria|   Jones|
+---------+--------+

+---------+--------+
|firstname|lastname|
+---------+--------+
|    James|   Smith|
|  Michael|    Rose|
|   Robert|Williams|
|    Maria|   Jones|
+---------+--------+

